# 📊 Movie Analytics Data Pipeline


---


In this task, you will build an ***end-to-end data analytics pipeline*** using ***Python and SQL*** based on real movie ratings data. You will ingest raw datasets, clean and transform them into analytics-ready tables, and answer business-style questions about movie popularity and user behavior. The task mirrors real data engineering workflows that support analytics and data science teams.

Skills Focus: SQL, data engineering fundamentals, Python + SQL integration, analytics thinking

## 🎬 Use Case: Movie Analytics Platform


---


You are working as a Data Engineer for a movie analytics platform.
The platform wants insights into:

* Movie popularity

* User engagement

* Rating trends over time

The Raw data is collected from users and stored in CSV files. Your job is to prepare this data so analysts can easily query and analyze it.

## 📂 Dataset


---

You will use the MovieLens Dataset, a widely used public dataset for recommender systems and analytics.

📥 Dataset Link (CSV format)

MovieLens – Movies & Ratings (Kaggle)
👉 https://www.kaggle.com/datasets/ayberkural/movielens-movie-csv-and-rating-csv

You will mainly work with:


* movies.csv
* ratings.csv




## 🗂️ Dataset Overview


---


1️⃣ movies.csv:

          |   movieId   |   title   |   genres   |

2️⃣ ratings.csv:

          |   userId   |   movieId   |   rating   |   timestamp   |   

## 🛠️ Task Breakdown

###🔹 Part 1: Data Ingestion & Cleaning


---


✅ Task 1.1 – Movie Dimension

* Create a movie dimension table:

* One row per movie

* Split genres into a standardized format if needed

* Ensure clean and consistent movie titles

📌 Output table: dim_movies

In [1]:
# your code here
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
# reading movies and ratings from the drive
file_path = '/content/drive/MyDrive/movies/archive'
movies = file_path + '/movie.csv'
ratings = file_path + '/rating.csv'
movies_df = pd.read_csv(movies)
ratings_df = pd.read_csv(ratings)

In [3]:
#Create a database file:
import sqlite3
movie_conn = sqlite3.connect("movies_dw.db")
movie_cursor = movie_conn.cursor()


In [4]:
# Create the Dimension Table
movie_cursor.execute("""
CREATE TABLE IF NOT EXISTS dim_movie (
    movie_key INTEGER PRIMARY KEY AUTOINCREMENT,
    movie_id INTEGER UNIQUE,
    title TEXT,
    genres TEXT
);
""")

movie_conn.commit()

In [5]:
#Load Data From Pandas → SQLite
movies_df[['movieId','title','genres']] \
    .rename(columns={'movieId':'movie_id'}) \
    .to_sql('dim_movie', movie_conn, if_exists='append', index=False)


27278

In [6]:
#Verify the Result
movie_cursor.execute("SELECT * FROM dim_movie LIMIT 5;")
movie_cursor.fetchall()

[(1, 1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
 (2, 2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
 (3, 3, 'Grumpier Old Men (1995)', 'Comedy|Romance'),
 (4, 4, 'Waiting to Exhale (1995)', 'Comedy|Drama|Romance'),
 (5, 5, 'Father of the Bride Part II (1995)', 'Comedy')]

✅ Task 1.2 – User Dimension

Create a user dimension table:

* One row per user

* Derived from the ratings data

📌 Output table: dim_users

In [ ]:
# your code here

In [7]:
#Create the Dimension Table
movie_cursor.execute("""
CREATE TABLE dim_user (
    user_key INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    movie_id INTEGER,
    rating REAL,
    timestamp INTEGER
);
""")

movie_conn.commit()


In [8]:
# Load Unique Users from ratings_df
ratings_clean = ratings_df.rename(columns={
    'userId': 'user_id',
    'movieId': 'movie_id'
})

ratings_clean.to_sql('dim_user', movie_conn, if_exists='append', index=False)


20000263

In [9]:
#Verify
movie_cursor.execute("SELECT * FROM dim_user LIMIT 5;")
movie_cursor.fetchall()

[(1, 1, 2, 3.5, '2005-04-02 23:53:47'),
 (2, 1, 29, 3.5, '2005-04-02 23:31:16'),
 (3, 1, 32, 3.5, '2005-04-02 23:33:39'),
 (4, 1, 47, 3.5, '2005-04-02 23:32:07'),
 (5, 1, 50, 3.5, '2005-04-02 23:29:40')]

###🔹 Part 2: Fact Table & Transformations


---


✅ Task 2.1 – Ratings Fact Table

Create a fact table that:

* Joins users and movies

* Contains one row per rating

* Includes:

    * rating value

    * rating timestamp

📌 Output table: fact_ratings

In [ ]:
# your code here

In [10]:
#Create Fast rating table
movie_cursor.execute("""
CREATE TABLE fact_ratings (
    rating_key INTEGER PRIMARY KEY AUTOINCREMENT,

    -- User columns
    user_key INTEGER,
    user_id INTEGER,
    movie_id INTEGER,
    rating REAL,
    timestamp INTEGER,

    -- Movie columns
    movie_key INTEGER,
    title TEXT,
    genres TEXT
);
""")

movie_conn.commit()

In [11]:
movie_cursor.execute("""
INSERT INTO fact_ratings (
    user_key, user_id, movie_id, rating, timestamp,
    movie_key, title, genres
)
SELECT
    du.user_key,
    du.user_id,
    du.movie_id,
    du.rating,
    du.timestamp,
    dm.movie_key,
    dm.title,
    dm.genres
FROM dim_user du
JOIN dim_movie dm
    ON du.movie_id = dm.movie_id;
""")

movie_conn.commit()


In [12]:
#Verify
movie_cursor.execute("SELECT * FROM fact_ratings LIMIT 5;")
movie_cursor.fetchall()

[(1,
  1,
  1,
  2,
  3.5,
  '2005-04-02 23:53:47',
  2,
  'Jumanji (1995)',
  'Adventure|Children|Fantasy'),
 (2,
  2,
  1,
  29,
  3.5,
  '2005-04-02 23:31:16',
  29,
  'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
  'Adventure|Drama|Fantasy|Mystery|Sci-Fi'),
 (3,
  3,
  1,
  32,
  3.5,
  '2005-04-02 23:33:39',
  32,
  'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
  'Mystery|Sci-Fi|Thriller'),
 (4,
  4,
  1,
  47,
  3.5,
  '2005-04-02 23:32:07',
  47,
  'Seven (a.k.a. Se7en) (1995)',
  'Mystery|Thriller'),
 (5,
  5,
  1,
  50,
  3.5,
  '2005-04-02 23:29:40',
  50,
  'Usual Suspects, The (1995)',
  'Crime|Mystery|Thriller')]

✅ Task 2.2 – Time Attributes

Enhance the fact table with:

    rating_date
    

---

    rating_month


---


    rating_year

In [ ]:
#Your code here

In [13]:
# adding date , month and year columns to fact_ratings
movie_cursor.execute("""
ALTER TABLE fact_ratings
ADD COLUMN rating_date TEXT
""")
movie_cursor.execute("""
ALTER TABLE fact_ratings
ADD COLUMN rating_month INTEGER
""")
movie_cursor.execute("""
ALTER TABLE fact_ratings
ADD COLUMN rating_year INTEGER
""")

movie_conn.commit()


In [16]:
# Populate New Columns from timestamp
movie_cursor.execute("""
UPDATE fact_ratings
SET
    rating_date = DATE(timestamp),
    rating_month = CAST(STRFTIME('%m', timestamp) AS INTEGER),
    rating_year = CAST(STRFTIME('%Y', timestamp) AS INTEGER)
""")
movie_conn.commit()


In [17]:
#Verify
movie_cursor.execute("SELECT * FROM fact_ratings LIMIT 5;")
movie_cursor.fetchall()

[(1,
  1,
  1,
  2,
  3.5,
  '2005-04-02 23:53:47',
  2,
  'Jumanji (1995)',
  'Adventure|Children|Fantasy',
  '2005-04-02',
  4,
  2005),
 (2,
  2,
  1,
  29,
  3.5,
  '2005-04-02 23:31:16',
  29,
  'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
  'Adventure|Drama|Fantasy|Mystery|Sci-Fi',
  '2005-04-02',
  4,
  2005),
 (3,
  3,
  1,
  32,
  3.5,
  '2005-04-02 23:33:39',
  32,
  'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
  'Mystery|Sci-Fi|Thriller',
  '2005-04-02',
  4,
  2005),
 (4,
  4,
  1,
  47,
  3.5,
  '2005-04-02 23:32:07',
  47,
  'Seven (a.k.a. Se7en) (1995)',
  'Mystery|Thriller',
  '2005-04-02',
  4,
  2005),
 (5,
  5,
  1,
  50,
  3.5,
  '2005-04-02 23:29:40',
  50,
  'Usual Suspects, The (1995)',
  'Crime|Mystery|Thriller',
  '2005-04-02',
  4,
  2005)]

###🔹 Part 3: Analytics Queries (SQL)

Write SQL queries to answer the following:

1.   Top 10 highest-rated movies (minimum number of ratings required)
2.   Most rated movies overall
3.   Average rating per genre
4.   Monthly rating activity trend
5.   Rank users by number of ratings submitted (window functions)

In [ ]:
#Your code here

In [18]:
#Top 10 highest-rated movies
top_movies = pd.read_sql("""
SELECT
    movie_id,
    title,
    COUNT(*) AS num_ratings,
    ROUND(AVG(rating), 2) AS avg_rating
FROM fact_ratings
GROUP BY movie_id, title
HAVING COUNT(*) >= 50
ORDER BY avg_rating DESC
LIMIT 10
""", movie_conn)

top_movies


,movie_id,title,num_ratings,avg_rating
0,318,"Shawshank Redemption, The (1994)",63366,4.45
1,858,"Godfather, The (1972)",41355,4.36
2,50,"Usual Suspects, The (1995)",47006,4.33
3,527,Schindler's List (1993),50054,4.31
4,1221,"Godfather: Part II, The (1974)",27398,4.28
5,904,Rear Window (1954),17449,4.27
6,2019,Seven Samurai (Shichinin no samurai) (1954),11611,4.27
7,912,Casablanca (1942),24349,4.26
8,922,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),6525,4.26
9,7502,Band of Brothers (2001),4305,4.26


In [19]:
#Most rated movies overall
most_rated_movies = pd.read_sql("""
SELECT
    movie_id,
    title,
    COUNT(*) AS num_ratings
FROM fact_ratings
GROUP BY movie_id, title
ORDER BY num_ratings DESC
LIMIT 10
""", movie_conn)

most_rated_movies


,movie_id,title,num_ratings
0,296,Pulp Fiction (1994),67310
1,356,Forrest Gump (1994),66172
2,318,"Shawshank Redemption, The (1994)",63366
3,593,"Silence of the Lambs, The (1991)",63299
4,480,Jurassic Park (1993),59715
5,260,Star Wars: Episode IV - A New Hope (1977),54502
6,110,Braveheart (1995),53769
7,589,Terminator 2: Judgment Day (1991),52244
8,2571,"Matrix, The (1999)",51334
9,527,Schindler's List (1993),50054


In [20]:
#Average Rating per Genre
avg_genre_rating_sql = pd.read_sql("""
WITH RECURSIVE genre_split(movie_id, rating, genre, rest) AS (
    SELECT
        movie_id,
        rating,
        CASE
            WHEN instr(genres, '|') > 0 THEN substr(genres, 1, instr(genres, '|') - 1)
            ELSE genres
        END AS genre,
        CASE
            WHEN instr(genres, '|') > 0 THEN substr(genres, instr(genres, '|') + 1)
            ELSE NULL
        END AS rest
    FROM fact_ratings
    WHERE genres IS NOT NULL

    UNION ALL

    SELECT
        movie_id,
        rating,
        CASE
            WHEN instr(rest, '|') > 0 THEN substr(rest, 1, instr(rest, '|') - 1)
            ELSE rest
        END AS genre,
        CASE
            WHEN instr(rest, '|') > 0 THEN substr(rest, instr(rest, '|') + 1)
            ELSE NULL
        END AS rest
    FROM genre_split
    WHERE rest IS NOT NULL
)
SELECT
    genre,
    ROUND(AVG(rating), 2) AS avg_rating,
    COUNT(*) AS num_ratings
FROM genre_split
GROUP BY genre
ORDER BY avg_rating DESC;
""", movie_conn)

avg_genre_rating_sql


,genre,avg_rating,num_ratings
0,Film-Noir,3.97,216689
1,War,3.81,1048618
2,Documentary,3.74,244619
3,Drama,3.67,8857853
4,Crime,3.67,3298335
5,Mystery,3.66,1557282
6,IMAX,3.66,492366
7,Animation,3.62,1140476
8,Western,3.57,423714
9,Musical,3.56,870915


In [21]:
# Monthly Rating Activity Trend
monthly_trend = pd.read_sql("""
SELECT
    rating_year,
    rating_month,
    COUNT(*) AS num_ratings,
    ROUND(AVG(rating), 2) AS avg_rating
FROM fact_ratings
GROUP BY rating_year, rating_month
ORDER BY rating_year, rating_month;
""", movie_conn)

monthly_trend



,rating_year,rating_month,num_ratings,avg_rating
0,1995,1,4,3.75
1,1996,1,42,4.17
2,1996,2,453,3.60
3,1996,3,9314,3.91
4,1996,4,52544,3.80
...,...,...,...,...
226,2014,11,79148,3.56
227,2014,12,79644,3.52
228,2015,1,107750,3.45
229,2015,2,88458,3.53


In [22]:
#Rank users by number of ratings submitted
user_ranking = pd.read_sql("""
SELECT
    user_id,
    COUNT(*) AS num_ratings,
    RANK() OVER (ORDER BY COUNT(*) DESC) AS user_rank
FROM fact_ratings
GROUP BY user_id
ORDER BY user_rank;
""", movie_conn)

user_ranking.head(10)


,user_id,num_ratings,user_rank
0,118205,9254,1
1,8405,7515,2
2,82418,5646,3
3,121535,5520,4
4,125794,5491,5
5,74142,5447,6
6,34576,5356,7
7,131904,5330,8
8,83090,5169,9
9,59477,4988,10


📦 Deliverables:

You should submit one Jupyter notebook:

📓 movie_analytics_pipeline.ipynb

The notebook must include:

* Data loading & cleaning steps

* SQL table creation and transformations

* Analytics queries and results

* Short markdown explanations for each section

✅ This Task Prepares You For:

* Analytics engineering & data engineering roles

* SQL-heavy technical interviews

* Real-world Python + SQL workflows

* Building analytics-ready datasets for data science